<a href="https://colab.research.google.com/github/JHyunjun/TF2.0_LSTM/blob/main/Bidirectional_LSTM_tf2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Created by Hyunjun, JANG
# limitsinx.tistory.com

import numpy as np
import pandas as pd
import tensorflow as tf
import pandas_datareader.data as pdr
import matplotlib.pyplot as plt
import datetime
tf.random.set_seed(1)


#Data Load
df_train = pd.read_csv('/dir')
df_test = pd.read_csv('/dir')
train_data = df_train.iloc[:,:]
test_data = df_test.iloc[:,:]
print(train_data)
print(test_data)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Normalization
def MinMaxScaler(data):
  denom = np.max(data,0) - np.min(data,0)
  nume = data-np.min(data,0)
  return nume/denom

def back_MinMax(data,max,min):
  diff = max-min
  back = data * diff + min
  return back

In [ ]:
#Hyper parameters
seqLength = 5
dataDim = df_test.shape[1]
outputDim = 1
hiddenDim = 10
lr = 0.001
iterations = 10
batch_size = 8

trainSize = int(len(train_data))
trainSet = train_data[0:trainSize]
testSize = int(len(test_data))
testSet = test_data[0:testSize]
testSet_for_result = testSet

trainSet_max = np.max(trainSet)
trainSet_min = np.min(trainSet)
testSet_max = np.max(testSet)
testSet_min = np.min(testSet)
trainSet_max = (trainSet_max[dataDim-1])
trainSet_min = (trainSet_min[dataDim-1])
testSet_max = (testSet_max[dataDim-1])
testSet_min = (testSet_min[dataDim-1])


trainSet = MinMaxScaler(trainSet)
testSet = MinMaxScaler(testSet)

print(trainSet.shape) # (5807,5)
print(testSet.shape) # (10882,5)

In [ ]:
#Data Windowing
def sliding_window(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range(0, len(time_series) - seq_length): 
        print(i)
        _x = time_series.iloc[i:i + seq_length, : -1 ]
        _y = time_series.iloc[i + seq_length, [-1]]  
        print(_x, "->", _y)
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

trainX, trainY = sliding_window(trainSet, seqLength)
testX, testY = sliding_window(testSet, seqLength)


In [ ]:
# Stacked LSTM Modeling
# Return Sequences is very important for stacking
# This means weather output is Sequence of this layer or all of sequences
# Activation function "tanh" : Glorot normal initializer
# Activation function "reLU" : He Initiailizer
from tensorflow import keras
from tensorflow.keras import layers

initializer = tf.keras.initializers.GlorotNormal()
#tf.keras.layers.Bidirectional(merge_mode='concat', weights=None, backward_layer=None)
input_dim = trainSet.shape[1]-1
model = keras.Sequential() #Bidirectional LSTM

model.add(layers.Bidirectional(layers.LSTM(units = hiddenDim*4, activation='relu',input_shape = (seqLength,input_dim), return_sequences = False, kernel_initializer = initializer)))
model.add(layers.Dense(outputDim, activation='linear'))
adam = keras.optimizers.Adam(lr = lr)
model.compile(optimizer=adam, loss='mse')

model2 = keras.Sequential() # OneDirection LSTM
model2.add(layers.LSTM(units = hiddenDim*2, activation='relu',input_shape = (seqLength,input_dim), return_sequences = False, kernel_initializer = initializer))
model2.add(layers.Dense(outputDim, activation='linear'))
adam = keras.optimizers.Adam(lr = lr)
model2.compile(optimizer=adam, loss='mse')

In [ ]:
# Training
hist = model.fit(trainX, trainY, epochs = iterations, batch_size = batch_size)
hist2 = model2.fit(trainX,trainY, epochs = iterations, batch_size = batch_size)

print("hist (Bidirectional) : ", hist)
print("hist (One way) : ", hist2)

In [ ]:
# Evaluate Testset
res = model.evaluate(testX, testY, batch_size = batch_size)
print("Bidirectional : ",res)
#res_batch_size1 = model.evaluate(testX, testY, batch_size = 1) #Very Slow

res2 = model2.evaluate(testX, testY, batch_size = batch_size)
print("One way : ",res)

In [ ]:
# Estimating - BD
xhat = testX
yhat = model.predict(xhat)
print("BD",yhat.shape)
#print("Bidirectional : {}".format(np.sqrt(np.average((yhat - testY)**2))))

#Estimating -OW
yhat2 = model2.predict(xhat)
print("OW",yhat2)
print(yhat2.shape)

#Error
Err = yhat-yhat2
print("ERR:",Err)
print(Err.shape)

In [ ]:
#Estimating
estimated = back_MinMax(yhat,testSet_max,testSet_min) 
real = testSet_for_result.iloc[:,-1] # estimated + seqLength

estimated2 = back_MinMax(yhat2, testSet_max, testSet_min)
Error = estimated - estimated2

In [ ]:
plt.figure()
plt.plot(estimated[:len(estimated)],label = "Estimated by Bidirectional LSTM")
plt.plot(estimated2[:len(estimated2)],label = "Estimated by One way LSTM")

plt.plot(real[:len(real)], label = "Real")
plt.legend(prop = {'size':10})

plt.figure()
plt.plot(Error[:len(Error)],label = "Difference : Bidirectional - One way")